In [1]:
import vertexai
from vertexai.language_models import TextGenerationModel
import pandas as pd

2024-09-09 13:51:40.028725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_csv(r'./input/CPR_Aug_2024.csv')

In [3]:
vertexai.init(project="ford-180395bd732cdd9af050c1f7", location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 5,
    "temperature": 0.2,
    "top_p": 0.8,
    "top_k": 1
}
model = TextGenerationModel.from_pretrained("text-bison@001")

In [4]:
prompt_train ="""Define the sentiment for the text below?
Options:
- positive
- negative
- neutral

If the text contains single word without any context then assign sentiment as neutral.

Text: It is american company that cares for the people.
sentiment: positive

Text: I have heard that they are making high quality products that are affordable.
sentiment: positive

Text: Ford has agreed to increase its workforce salaries.
sentiment: positive

Text: Problems with a F-150 Large Engine
sentiment: negative

Text: I have family that own Ford cars and they arent happy with the service they get in their cars
sentiment: negative

Text: The new electric vehicules has not doing like they were expected.
sentiment: negative

Text: Ford has partnered with Germanys Volkswagen Group to develop electric vehicle technology.
sentiment: neutral

Text: It was an advertisement for new vehicles
sentiment: neutral

Text: Don't remember
sentiment: neutral

Text: The new Ford F-150 Lightning
sentiment: neutral

Text: """

prompt_senti = "\nsentiment: "


In [5]:
import time
def split_data_senti(sub_data):
    sentiment=[]
    for i in range(0,len(sub_data)):
        prompt_test = sub_data['text'].iloc[i]
        prompt_test_s = str(prompt_test)
        prompt = prompt_train + prompt_test_s + prompt_senti
        response = model.predict(prompt,**parameters)
        #print(f"{response.text}")
        sentiment.append(response.text)
        time.sleep(2)
    
    final = pd.concat([sub_data,pd.DataFrame(sentiment, columns=['sentiment'])], axis=1)
    return final

In [6]:
output_few = split_data_senti(data)

In [7]:
output_few

,id,text,month,year,sentiment
0,R_1QXxniNV3YpqUPq,It was about new vehicle,Aug,2024,neutral
1,R_3DhVOZ3PTKZkWgp,Yes I have seen commercials for ford,Aug,2024,neutral
2,R_5Ea54NwgY2tkvR6,buy a ford product,Aug,2024,neutral
3,R_1YEbWB9AKWFLJnK,largest employer in usa,Aug,2024,neutral
4,R_1co47Ec7FvuwRpO,Fords recall of F-150 vehicles,Aug,2024,negative
...,...,...,...,...,...
467,R_6nCY5c1j1ZGUX9S,that it was exapanding,Aug,2024,neutral
468,R_3t9fVWP70zFU9iP,It has a good fit for motor that make a good f...,Aug,2024,positive
469,R_1n0P0NNv5Ijvesp,Great,Aug,2024,positive
470,R_3P3eupNFhLpNxFT,No trucks out,Aug,2024,negative


In [8]:
output_few.to_csv('./output/sentiment_cpr_fewshot_Aug_2024.csv')